In [1]:
import numpy as np
import os
import pandas as pd
import sys

# Config

In [2]:
HYPER_OPT = True 
HYPER_OPT_TIME = 8*60*60
USE_ORIGINAL_DATA = False 
SUBMIT_TO_KAGGLE = False
COMPUTE_NONLOG = False
USE_NONLOG = False

In [3]:
# Define the base directory (where the notebook is running)
base_dir = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
data_dir = os.path.join(base_dir, "data")
model_dir = os.path.join(base_dir, "models")
notebooks_dir = os.path.join(base_dir, "notebooks")
sys.path.append(base_dir)

# Data Extraction

In [4]:
from axyom_utilities.data_extraction import extract_data
from axyom_utilities.training import train_model_cv
from axyom_utilities.preprocessing import * 
from toolz import pipe
import joblib

# oof_log = pd.read_csv(os.path.join(notebooks_dir, "Catboost_nonlog\\oof_preds_log.csv"))
# oof_nonlog = pd.read_csv(os.path.join(notebooks_dir, "Catboost_nonlog\\oof_preds_nonlog.csv"))

# test_log = pd.read_csv(os.path.join(notebooks_dir, "Catboost_nonlog\\test_preds_log.csv"))
# test_nonlog = pd.read_csv(os.path.join(notebooks_dir, "Catboost_nonlog\\test_preds_nonlog.csv"))

# X_train["oof_nonlog"] = oof_nonlog + np.random.normal(0, 0.01, oof_log.shape) # noise
# X_test["oof_nonlog"] = test_nonlog + np.random.normal(0, 0.01, oof_log.shape)

# X_test["oof_log"] = test_log
# X_train["oof_log"] = oof_log

In [5]:
X_train, y_train, X_test, X_orig, y_orig = extract_data(data_dir, USE_ORIGINAL_DATA, log_transform=True)
y_train_nonlog = np.expm1(y_train)

if USE_NONLOG:
    if COMPUTE_NONLOG:
        nonlog_oof_preds, nonlog_test_preds = get_oof(X_train, y_train_nonlog, X_test)
    else:
        nonlog_oof_preds, nonlog_test_preds = joblib.load('nonlog_feature/cat_non_loged.pkl')
    
    X_train["nonlog"] = nonlog_oof_preds
    X_test["nonlog"] = nonlog_test_preds

# Data Cleaning

In [6]:
from axyom_utilities.preprocessing import freq_encode, preprocess_dates, clean_categorical, frequency_encode
from toolz import pipe

def feature_engineering(df):
    
    # df = freq_encode(df)
    
    df['MissingHealth'] = df['Health Score'].isna().astype(int)
    df['Claims v Duration'] = (df['Previous Claims'] / df['Insurance Duration']).replace([np.inf, -np.inf], np.nan)
    df['Health vs Claims'] = (df['Health Score'] / df['Previous Claims']).replace([np.inf, -np.inf], np.nan)
    
    def duplicate_int(col, df):
        df[col + ' Integer'] = df[col].apply(lambda x: int(x) if pd.notna(x) else x)
        df[col] = df[col].fillna('None').astype('category')
        
        return df
    
    df = duplicate_int('Health Score', df)
    df = duplicate_int('Credit Score', df)
    df = duplicate_int('Previous Claims', df)
    df = duplicate_int('Annual Income', df)
    
    df['MissingValuesCount'] = df.isna().sum(axis=1)
    
    # df = freq_encode(df, 'Customer Feedback')
    # df = freq_encode(df, 'Marital Status')
    
    return df

preprocessor = lambda df: pipe(df,\
    preprocess_dates,
    clean_categorical,
    feature_engineering,  
)

X_train = preprocessor(X_train)
X_test = preprocessor(X_test)
if USE_ORIGINAL_DATA:
    X_orig = preprocessor(X_orig)
    
frequency_encode(X_train, X_test, drop_org=False) # data leaking but acceptable in this context


(          Age  Gender Annual Income Marital Status  Number of Dependents  \
 id                                                                         
 0        19.0  Female       10049.0        Married                   1.0   
 1        39.0  Female       31678.0       Divorced                   3.0   
 2        23.0    Male       25602.0       Divorced                   3.0   
 3        21.0    Male      141855.0        Married                   2.0   
 4        21.0    Male       39651.0         Single                   1.0   
 ...       ...     ...           ...            ...                   ...   
 1199995  36.0  Female       27316.0        Married                   0.0   
 1199996  54.0    Male       35786.0       Divorced                   NaN   
 1199997  19.0    Male       51884.0       Divorced                   0.0   
 1199998  55.0    Male          None         Single                   1.0   
 1199999  21.0  Female          None       Divorced                   0.0   

# HyperOpt

In [12]:
large_values = X_train.select_dtypes(include=["number"]).max() > 1e5
print(large_values)

Age                        False
Number of Dependents       False
Vehicle Age                False
Insurance Duration         False
Month                      False
Day                        False
Week                       False
Weekday                    False
DaySin                     False
DayCos                     False
WeekdaySin                 False
WeekdayCos                 False
DaysSinceStart             False
MissingHealth              False
Claims v Duration          False
Health vs Claims           False
Health Score Integer       False
Credit Score Integer       False
Previous Claims Integer    False
Annual Income Integer       True
MissingValuesCount         False
Gender_freq                 True
Annual Income_freq         False
Marital Status_freq         True
Education Level_freq        True
Occupation_freq             True
Health Score_freq           True
Location_freq               True
Policy Type_freq            True
Previous Claims_freq        True
Credit Sco

In [13]:
# Find columns with inf or -inf values
inf_columns = X_train.columns[X_train.isin([np.inf, -np.inf]).any()]
print("Columns with inf or -inf values:", inf_columns)

# Find columns with NaN values
nan_columns = X_train.columns[X_train.isna().any()]
print("Columns with NaN values:", nan_columns)


Columns with inf or -inf values: Index([], dtype='object')
Columns with NaN values: Index(['Age', 'Number of Dependents', 'Vehicle Age', 'Insurance Duration',
       'Claims v Duration', 'Health vs Claims', 'Health Score Integer',
       'Credit Score Integer', 'Previous Claims Integer',
       'Annual Income Integer'],
      dtype='object')


In [14]:
from axyom_utilities.hyperparameter_tuning import XGBoostTuner

if HYPER_OPT:
    tuner = XGBoostTuner(X_train, y_train, HYPER_OPT_TIME)
    best_params = tuner.tune()
    tuner.plot()
else:
    best_params = {
        "iterations":3000,
        "learning_rate":0.05,
        "depth":6,
        "eval_metric":"RMSE",
        "random_seed":42,
        "verbose":200,
        "task_type":'GPU',
        "l2_leaf_reg" :0.7,
    }

[I 2024-12-22 02:46:22,793] Using an existing study with name 'xgboost' instead of creating a new one.


Training fold 1...
Fold 1 RMSE: 1.0473
Training fold 2...
Fold 2 RMSE: 1.0476
Training fold 3...
Fold 3 RMSE: 1.0456
Training fold 4...
Fold 4 RMSE: 1.0479
Training fold 5...


[I 2024-12-22 03:12:45,893] Trial 3 finished with value: 1.0471181537825136 and parameters: {'learning_rate': 0.05433350003437882, 'max_depth': 14, 'min_child_weight': 1.509804001525835, 'subsample': 0.8367148343488449, 'colsample_bytree': 0.9708747744678036, 'gamma': 0.06351598517383816, 'lambda': 0.04511809923805299, 'alpha': 0.0015676574662308934}. Best is trial 3 with value: 1.0471181537825136.


Fold 5 RMSE: 1.0471
Mean CV RMSE: 1.0471 ± 0.0008
Training fold 1...


[W 2024-12-22 04:52:41,137] Trial 4 failed with parameters: {'learning_rate': 0.001768215377734423, 'max_depth': 12, 'min_child_weight': 0.008728853020978604, 'subsample': 0.8464925012884582, 'colsample_bytree': 0.5157352018243503, 'gamma': 3.5042237608299063, 'lambda': 0.014890607747597583, 'alpha': 1.2529040614881077} because of the following error: XGBoostError('bad allocation').
Traceback (most recent call last):
  File "c:\Users\Axyom\anaconda3\envs\kaggle_env\lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "c:\Users\Axyom\Kaggle\PG S4e12 - Regression Insurance\axyom_utilities\hyperparameter_tuning.py", line 27, in <lambda>
    objective = lambda trial: self.generic_objective(trial, model_generator)
  File "c:\Users\Axyom\Kaggle\PG S4e12 - Regression Insurance\axyom_utilities\hyperparameter_tuning.py", line 42, in generic_objective
    results = train_model_cv(
  File "c:\Users\Axyom\Kaggle\PG S4e12 - Regression Insura

XGBoostError: bad allocation

In [ ]:
best_params

In [10]:
best_params["n_iterators"] = 10000

# Model Training

In [ ]:
import json
from axyom_utilities.wrappers import XGBRegressorWrapper

model = XGBRegressorWrapper(**best_params)

# Use the train_model function to train and evaluate the model
results = train_model_cv(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    X_orig=X_orig,
    early_stopping_rounds=100,
    cv_splits=5
)

In [ ]:
# models = results['models']

# # Get feature importance
# feature_importances = models[3].catboost_model_.get_feature_importance(type='PredictionValuesChange')

# # Map to feature names
# feature_importance_df = pd.DataFrame({
#     'Feature': X_train.columns,
#     'Importance': feature_importances
# }).sort_values(by='Importance', ascending=False)

# print(feature_importance_df)

In [14]:
mean_score = np.mean(results["cv_scores"])
std_score = np.std(results["cv_scores"])

# Saving

In [15]:
import joblib

results_dir = f"{mean_score:.4f}+-{std_score:.4f}"
os.makedirs(results_dir, exist_ok=True)

# Save OOF predictions as a CSV file
oof_preds_df = pd.DataFrame({"oof_preds": results["oof_preds"]})
oof_preds_df.to_csv(os.path.join(results_dir, "oof_preds.csv"), index=False)

test_preds_df = pd.DataFrame({"test_preds": results["test_preds"]})
test_preds_df.to_csv(os.path.join(results_dir, "test_preds.csv"), index=False)

#joblib.dump(results["models"], "models.pkl")

# Submission

In [16]:
y_pred = np.expm1(results['test_preds'])

submission = pd.DataFrame({
    'id': X_test.index,  
    'Premium Amount': y_pred
})

FILE_PATH = os.path.join(results_dir, f"XGBoost_v2_{mean_score:.4f}+-{std_score:.4f}.csv")

submission.to_csv(FILE_PATH, index=False)

In [17]:
if True: 
    # Define your message and file paths
    COMP_NAME = "playground-series-s4e12"

    SUBMIT_MESSAGE = "{mean_score:.4f}+-{std_score:.4f}"

    # Submit to Kaggle
    os.system(f'kaggle competitions submit -c {COMP_NAME} -f {FILE_PATH} -m "{SUBMIT_MESSAGE}"')